# El problema de trabajar con Valores Faltantes mediante Modelos

## Librerias

In [1]:
import matplotlib.pyplot as plt 
import numpy as np  
import pandas as pd 
import scipy.stats  # trabjar con estadisticos
import seaborn as sns
import statsmodels.api as sm 
import statsmodels.formula.api as smf

## Configuracion de las graficas

In [2]:
%matplotlib inline

# tamaño de los graficos
sns.set(
    rc={
        'figure.figsize':(8, 6)
    }
)

# estilo de los graficos
sns.set_style('whitegrid')

## Cargar los Datos